In [2]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense ,Activation
from keras.optimizers import Adam

columns = (['duration','protocol_type','service','flag',
            'src_bytes','dst_bytes','land','wrong_fragment','urgent','hot'
,'num_failed_logins','logged_in','num_compromised','root_shell',
            'su_attempted','num_root','num_file_creations'
,'num_shells','num_access_files','num_outbound_cmds',
            'is_host_login','is_guest_login','count','srv_count','serror_rate'
,'srv_serror_rate','rerror_rate','srv_rerror_rate',
            'same_srv_rate','diff_srv_rate','srv_diff_host_rate',
            'dst_host_count','dst_host_srv_count'
,'dst_host_same_srv_rate','dst_host_diff_srv_rate',
            'dst_host_same_src_port_rate','dst_host_srv_diff_host_rate',
            'dst_host_serror_rate'
,'dst_host_srv_serror_rate','dst_host_rerror_rate',
            'dst_host_srv_rerror_rate','outcome','level'])


df_train = pd.read_csv("/home/yesser/Downloads/intrusion/KDDTrain+.txt", header=None)
df_test = pd.read_csv("/home/yesser/Downloads/intrusion/KDDTest+.txt",  header=None)

df = pd.concat([df_train, df_test], ignore_index=True)
df.columns = columns
df.head()

2024-02-15 00:48:38.671921: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,outcome,level
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21


In [ ]:
df.shape

In [ ]:
labels = []
for i in df["outcome"]:
    if i == "normal":
        labels.append(0)
    else: labels.append(1)
        
labels = np.array(labels)

In [ ]:
df = df.drop(["outcome"],axis=1)

In [ ]:
def make_dict(l):
    d = {}
    s = set(l)
    for e,i in enumerate(s):
        d[i] = e
    return d
        

In [ ]:
dict_protocol = make_dict(df['protocol_type'])
dict_service = make_dict(df['service'])
dict_flag = make_dict(df['flag'])

In [ ]:
dict_flag

In [ ]:
df['flag'] = df['flag'].map(dict_flag)
df['protocol_type'] = df['protocol_type'].map(dict_protocol)
df['service'] = df['service'].map(dict_service)

df_test['flag'] = df['flag'].map(dict_flag)
df_test['protocol_type'] = df['protocol_type'].map(dict_protocol)
df['service'] = df['service'].map(dict_service)

In [ ]:
df.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

l = ['protocol_type', 'service', 'flag','level']

scaler = MinMaxScaler()

for col in df.columns:
    if col not in l:
        scaler.fit(np.array(df[col]).reshape(-1, 1))
        df[col] = scaler.transform(np.array(df[col]).reshape(-1, 1))
        df_test[col] = scaler.transform(np.array(df_test[col]).reshape(-1, 1))



In [ ]:
df.head()

In [83]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [84]:
model = Sequential()
model.add(Dense(128, input_shape=(df_train_data.shape[1],)))
model.add(Activation("relu"))
model.add(Dense(64))
model.add(Activation("relu"))
model.add(Dense(32))
model.add(Dense(1))
model.add(Activation("sigmoid"))

model.compile(optimizer=Adam(),
              loss="binary_crossentropy",
              metrics=["accuracy"])

model.summary()

history = model.fit(X_train, y_train, epochs=5, batch_size=64, 
                    validation_data=(X_val, y_val))


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 128)               5504      
                                                                 
 activation_7 (Activation)   (None, 128)               0         
                                                                 
 dense_11 (Dense)            (None, 64)                8256      
                                                                 
 activation_8 (Activation)   (None, 64)                0         
                                                                 
 dense_12 (Dense)            (None, 32)                2080      
                                                                 
 dense_13 (Dense)            (None, 1)                 33        
                                                                 
 activation_9 (Activation)   (None, 1)                

2023-05-16 18:55:26.826998: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 31936800 exceeds 10% of free system memory.


Epoch 1/5
1486/1486 [==============================] - 19s 10ms/step - loss: 0.0735 - accuracy: 0.9737 - val_loss: 0.0437 - val_accuracy: 0.9851
Epoch 2/5
1486/1486 [==============================] - 5s 3ms/step - loss: 0.0343 - accuracy: 0.9875 - val_loss: 0.0305 - val_accuracy: 0.9901
Epoch 3/5
1486/1486 [==============================] - 5s 4ms/step - loss: 0.0278 - accuracy: 0.9897 - val_loss: 0.0257 - val_accuracy: 0.9910
Epoch 4/5
1486/1486 [==============================] - 5s 3ms/step - loss: 0.0242 - accuracy: 0.9912 - val_loss: 0.0213 - val_accuracy: 0.9926
Epoch 5/5
1486/1486 [==============================] - 5s 3ms/step - loss: 0.0205 - accuracy: 0.9922 - val_loss: 0.0178 - val_accuracy: 0.9930


In [100]:
met = model.evaluate(X_test, y_test)
print("Accuracy : ",met[1])
print("Loss : ",met[0])

929/929 [==============================] - 2s 2ms/step - loss: 0.0197 - accuracy: 0.9927
Accuracy :  0.9927282333374023
Loss :  0.01974855549633503


In [95]:
from sklearn.metrics import confusion_matrix

y_pred = []
for p in model.predict(X_test):
    y_pred.append(round(p[0]))

cm = confusion_matrix(y_test, y_pred)
cm

929/929 [==============================] - 2s 2ms/step


array([[15303,   147],
       [   69, 14185]])

In [97]:
"""
#saving a model

model_json = model.to_json()
with open("model99"+".json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model99"+".h5")
print("Sauvegarde")"""

Sauvegarde


In [103]:
"""
#loading a saved model

from tensorflow.keras.models import model_from_json

json_file = open("model99"+'.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("model99"+".h5")

loaded_model.compile(optimizer=Adam(),
              loss="binary_crossentropy",
              metrics=["accuracy"])

met = loaded_model.evaluate(X_test, y_test)
print("Accuracy : ",met[1])
print("Loss : ",met[0])"""

'\n#loading a saved model\n\nfrom tensorflow.keras.models import model_from_json\n\njson_file = open("model99"+\'.json\', \'r\')\nloaded_model_json = json_file.read()\njson_file.close()\nloaded_model = model_from_json(loaded_model_json)\nloaded_model.load_weights("model99"+".h5")\n\nloaded_model.compile(optimizer=Adam(),\n              loss="binary_crossentropy",\n              metrics=["accuracy"])\n\nmet = loaded_model.evaluate(X_test, y_test)\nprint("Accuracy : ",met[1])\nprint("Loss : ",met[0])'